<a href="https://colab.research.google.com/github/kbondar17/basic-flask-template/blob/master/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22TensorFlow_with_GPU%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install transformers
# !pip install torch
# !pip install streamlit
# !pip install diffusers
# !pip install datasets
#!pip install transformers[torch]
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [68]:
import os

os.getcwd()

'/content'

In [58]:
"""IMPORTS"""

from pprint import pprint
import numpy as np
from transformers import (AutoTokenizer,
                          AutoModel,
                          BertTokenizer,
                          BertModel,
                          AutoModelForSequenceClassification,
                          DataCollatorWithPadding,
                          TrainingArguments,
                          Trainer,
                          TextClassificationPipeline
                          )

from datasets import Dataset

from torch import tensor
import torch
import evaluate


In [47]:
"""Initialization"""

device = torch.device("cuda:0")

# bert = "distilbert-base-uncased-finetuned-sst-2-english"
bert = 'bert-base-uncased'
bert_tokenizer = AutoTokenizer.from_pretrained(bert)
bert_model = AutoModelForSequenceClassification.from_pretrained(bert, num_labels=2).to(device)

data_collator = DataCollatorWithPadding(tokenizer=bert_tokenizer)


In [ ]:
"""DATA"""

from datasets import load_dataset
raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [ ]:
"""функции"""

def tokenize_function(example):
    return bert_tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)


Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")


trainer = Trainer(
    bert_model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=bert_tokenizer,
    compute_metrics=compute_metrics,

)

trainer.train()



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.708801,0.838235,0.890000
2,0.289200,0.683612,0.862745,0.901060
3,0.166100,0.701232,0.865196,0.906303


TrainOutput(global_step=1377, training_loss=0.1829895498792417, metrics={'train_runtime': 201.6879, 'train_samples_per_second': 54.56, 'train_steps_per_second': 6.827, 'total_flos': 406183858377360.0, 'train_loss': 0.1829895498792417, 'epoch': 3.0})

In [64]:
raw_datasets['train']

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 3668
})

In [67]:
ds = Dataset.from_dict({"sentence1": ["The text to predict", "squirtle", "this is a huge building"], "sentence2": ["The text to predict", "fire", "this house is big"]})
tokenized_ds = ds.map(tokenize_function, batched=True)

trainer.predict(test_dataset=tokenized_ds)



Map:   0%|          | 0/3 [00:00<?, ? examples/s]

PredictionOutput(predictions=array([[-3.5419834,  3.6580787],
       [ 1.6541787, -1.9135098],
       [-3.3751163,  3.4057634]], dtype=float32), label_ids=None, metrics={'test_runtime': 0.0484, 'test_samples_per_second': 61.984, 'test_steps_per_second': 20.661})

In [ ]:
# predictions = trainer.predict(tokenized_datasets["validation"])
# keys 'index', 'label_ids', 'metrics', 'c'

# print(predictions.predictions.shape, predictions.label_ids.shape)

# print(dir(predictions))
# predictions.predictions[0]
# predictions.label_ids[0]

# # predictions.predictions
# preds = np.argmax(predictions.predictions, axis=-1)
# preds


# metric = evaluate.load("glue", "mrpc")
# metric.compute(predictions=preds, references=predictions.label_ids)
# tokenized_datasets['test'][0]
# tokenized_datasets['test'][0]['label']

# from datasets.arrow_dataset import Dataset


# type(tokenized_datasets["validation"])


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 21>:21                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3129 in predict                  │
│                                                                                                  │
│   3126 │   │   start_time = time.time()                                                          │
│   3127 │   │                                                                                     │
│   3128 │   │   eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else se  │
│ ❱ 3129 │   │   output = eval_loop(                                                               │
│   3130 │   │   │   test_dataloader, description="Prediction", ignore_keys=ignore_keys, metric_k  │
│   3131 │   │   )                                                                                 │
│   3132 │   │   total_batch_size = self.args.eval_batch_size * self.args.world_size               │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3235 in evaluation_loop          │
│                                                                                                  │
│   3232 │   │                                                                                     │
│   3233 │   │   observed_num_examples = 0                                                         │
│   3234 │   │   # Main evaluation loop                                                            │
│ ❱ 3235 │   │   for step, inputs in enumerate(dataloader):                                        │
│   3236 │   │   │   # Update the observed num examples                                            │
│   3237 │   │   │   observed_batch_size = find_batch_size(inputs)                                 │
│   3238 │   │   │   if observed_batch_size is not None:                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:633 in __next__           │
│                                                                                                  │
│    630 │   │   │   if self._sampler_iter is None:                                                │
│    631 │   │   │   │   # TODO(https://github.com/pytorch/pytorch/issues/76750)                   │
│    632 │   │   │   │   self._reset()  # type: ignore[call-arg]                                   │
│ ❱  633 │   │   │   data = self._next_data()                                                      │
│    634 │   │   │   self._num_yielded += 1                                                        │
│    635 │   │   │   if self._dataset_kind == _DatasetKind.Iterable and \                          │
│    636 │   │   │   │   │   self._IterableDataset_len_called is not None and \                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:677 in _next_data         │
│                                                                                                  │
│    674 │                                                                                         │
│    675 │   def _next_data(self):                                                                 │
│    676 │   │   index = self._next_index()  # may raise StopIteration                             │
│ ❱  677 │   │   data = self._dataset_fetcher.fetch(index)  # may raise StopIteration              │
│    678 │   │   if self._pin_memory:                        

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)



array([1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1,